In [2]:
pip install geopy


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd

In [4]:
uno_postman=pd.read_parquet("../processed/per_postman_pickles/postman_0c032cafea40c06fa6337701caf35607.parquet")

uno_postman.head()




,ds,postman_id,gps_time,lat,lng
0,412,0c032cafea40c06fa6337701caf35607,04-12 06:42:42,1.712865e+06,1.090405e+06
1,412,0c032cafea40c06fa6337701caf35607,04-12 06:43:02,1.712847e+06,1.090413e+06
2,412,0c032cafea40c06fa6337701caf35607,04-12 06:43:22,1.712858e+06,1.090411e+06
3,412,0c032cafea40c06fa6337701caf35607,04-12 06:43:42,1.712860e+06,1.090402e+06
4,412,0c032cafea40c06fa6337701caf35607,04-12 06:44:02,1.712846e+06,1.090406e+06


In [7]:
from geopy.geocoders import Nominatim
import math

# Function to convert BD09 (Baidu) to WGS-84
def bd09_to_wgs84(bd_lat, bd_lng):
    # Scale by 10,000 to convert BD09 to degrees
    bd_lat /= 10000
    bd_lng /= 10000

    x = bd_lng - 0.0065
    y = bd_lat - 0.006
    z = math.sqrt(x * x + y * y) - 0.00002 * math.sin(y * math.pi)
    theta = math.atan2(y, x) - 0.000003 * math.cos(x * math.pi)
    wgs_lng = z * math.cos(theta)
    wgs_lat = z * math.sin(theta)
    return wgs_lat, wgs_lng

# Given BD09 coordinates
bd_lat = 1.712865e+06  # Example values
bd_lng = 1.090405e+06  # Example values

# Convert to WGS-84
wgs_lat, wgs_lng = bd09_to_wgs84(bd_lat, bd_lng)

# Print converted coordinates
print(f"Converted Coordinates: Latitude={wgs_lat}, Longitude={wgs_lng}")

# Ensure valid latitude and longitude
if -90 <= wgs_lat <= 90 and -180 <= wgs_lng <= 180:
    # Initialize Nominatim API
    geolocator = Nominatim(user_agent="geoapiExercises")
    
    # Get location
    location = geolocator.reverse((wgs_lat, wgs_lng))
    
    # Extract city information
    if location and 'city' in location.raw['address']:
        city = location.raw['address']['city']
        print(f"The city is: {city}")
    else:
        print("City not found.")
    
    # Check if the city is one of the 5 major cities
    major_cities = ["Beijing", "Shanghai", "Guangzhou", "Shenzhen", "Chengdu"]
    if city in major_cities:
        print(f"{city} is one of the five major cities.")
    else:
        print(f"{city} is not one of the five major cities.")
else:
    print("Converted coordinates are out of bounds.")


Converted Coordinates: Latitude=171.2808382536806, Longitude=109.0334973726194
Converted coordinates are out of bounds.


In [4]:
def haversine_distance(lng1, lat1, lng2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees).
    Returns distance in kilometers.
    """
    # Convert decimal degrees to radians
    lng1, lat1, lng2, lat2 = map(np.radians, [lng1, lat1, lng2, lat2])
    
    # Haversine formula
    dlng = lng2 - lng1
    dlat = lat2 - lat1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlng/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    
    # Radius of Earth in kilometers
    r = 6371
    
    return c * r
# Calculate distance for each order
deli_hz['straight_line_distance_km'] = haversine_distance(
    deli_hz['accept_gps_lng'],
    deli_hz['accept_gps_lat'],
    deli_hz['delivery_gps_lng'],
    deli_hz['delivery_gps_lat']
)
# Get total straight line distance covered
total_distance = deli_hz['straight_line_distance_km'].sum()
print(f"Total straight line distance covered in HZ: {total_distance:.2f} km")

NameError: name 'deli_hz' is not defined

In [5]:
# search accross 5 delivery datasets to query about postman ids 

import pandas as pd
from pathlib import Path

def find_postman_in_delivery_files(postman_ids):
    """
    Search for postman/courier IDs across all delivery CSV files.
    
    Parameters:
    -----------
    postman_ids : list
        List of postman/courier IDs to search for
        
    Returns:
    --------
    dict : Dictionary mapping each postman_id to the file(s) where it was found
    """
    # Define the delivery files
    delivery_dir = Path("../LaDe/delivery")
    delivery_files = [
        "delivery_cq.csv",  # Chongqing
        "delivery_hz.csv",  # Hangzhou
        "delivery_jl.csv",  # Jilin
        "delivery_sh.csv",  # Shanghai
        "delivery_yt.csv"   # Yantai
    ]
    
    # Dictionary to store results
    results = {pid: [] for pid in postman_ids}
    
    print("Searching for postman IDs in delivery files...\n")
    
    # Search each file
    for file_name in delivery_files:
        file_path = delivery_dir / file_name
        print(f"Reading {file_name}...")
        
        try:
            # Read only the courier_id column to save memory
            df = pd.read_csv(file_path, usecols=['courier_id'])
            unique_couriers = set(df['courier_id'].unique())
            
            # Check which postman IDs are in this file
            for pid in postman_ids:
                if pid in unique_couriers:
                    results[pid].append(file_name)
                    print(f"  ✓ Found {pid} in {file_name}")
            
            del df  # Free memory
            
        except Exception as e:
            print(f"  ✗ Error reading {file_name}: {str(e)}")
    
    return results


def print_results(results):
    """Print the search results in a readable format"""
    print("\n" + "="*60)
    print("SEARCH RESULTS")
    print("="*60)
    
    for postman_id, files in results.items():
        if files:
            print(f"\nPostman ID: {postman_id}")
            print(f"  Found in: {', '.join(files)}")
        else:
            print(f"\nPostman ID: {postman_id}")
            print(f"  NOT FOUND in any file")
    
    print("\n" + "="*60)


# Example usage
if __name__ == "__main__":
    # Replace these with your actual postman IDs
    # These can be either numeric courier_ids or hashed postman_ids
    postman_ids_to_search = [
        "0ed1ca5e2616250dc64f3a817cb26422",      # Example numeric courier_id (from HZ data)
        "1cbb177ed5ce662364974b1e45300163",       # Example numeric courier_id (from CQ data)
        "3e744277d952051a650b5b32fc3d3a3e",     # Example numeric courier_id
        "7f398a8d971b763dc5b1b6d2409e856a",      # Example to test
    "97c2377e4c9f8d260162d8cb9c343433"      # Example to test
    ]
    
    # If you have hashed postman_ids like "0c032cafea40c06fa6337701caf35607"
    # you would need to use the trajectory data which maps postman_id to actual deliveries
    
    results = find_postman_in_delivery_files(postman_ids_to_search)
    print_results(results)
    
    # Return results as a DataFrame for easier analysis
    results_df = pd.DataFrame([
        {'postman_id': pid, 'file': file} 
        for pid, files in results.items() 
        for file in files
    ])
    
    if not results_df.empty:
        print("\nResults as DataFrame:")
        print(results_df)
    else:
        print("\nNo matches found for any postman ID")


Searching for postman IDs in delivery files...

Reading delivery_cq.csv...
Reading delivery_hz.csv...
Reading delivery_jl.csv...
Reading delivery_sh.csv...
Reading delivery_yt.csv...

SEARCH RESULTS

Postman ID: 0ed1ca5e2616250dc64f3a817cb26422
  NOT FOUND in any file

Postman ID: 1cbb177ed5ce662364974b1e45300163
  NOT FOUND in any file

Postman ID: 3e744277d952051a650b5b32fc3d3a3e
  NOT FOUND in any file

Postman ID: 7f398a8d971b763dc5b1b6d2409e856a
  NOT FOUND in any file

Postman ID: 97c2377e4c9f8d260162d8cb9c343433
  NOT FOUND in any file


No matches found for any postman ID
